# L4 - Обучение с подкреплением (reinforcement learning)

**Обучение с подкреплением** - один из способов машинного обучения, где некоторый агент обучается, взаимодействуя со средой. В отличие от обучения с учителем тут нет размеченных данных. Однако агент обладает набором допустимых действий, выполняя которые, он получает награду. Цель агента - максимизировать суммарную награду за какой-то период времени. Рассмотрим возможные постановки задачи.

Материал этой лабораторной основывается на этой прекрасной [книге](https://webdocs.cs.ualberta.ca/~sutton/book/bookdraft2016sep.pdf), которой вы может впоспользоваться.

## 1. Задача о многоруком бандите (multi-armed bandit problem)

### Постановка задачи

#### Неформальная
Пусть есть $N$ игровых автоматов. Сыграв на $i$-м автомате, агент получает награду, которая задается случайной величиной. Агент не знает, как устроены распределения, однако перед ним стоит задача максимизировать выигрыш сыграв $T$ раз.

#### Формальная
Пусть есть конечное множество возможных действий $A$. Для каждого действия $a \in A$ существует премия, которая определяется неизвестным распределением $p(r|a)$. Стратегия агента в момент $t$ - это некоторое вероятностное распределение на множестве всех действий $\pi_t(a)$.

Игра происходит следующим образом:
> 1. У агента есть некоторая начальная стратегия $\pi_1(a)$
> 2. В каждый момент времени $1 \leq t \leq T$:
> 3. Выибрает действией $a_t \sim \pi_t(a)$
> 4. Получает свою награду $r_t \sim p(r|a_t)$
> 5. Корректирует свою стратегию $\pi_t \rightarrow \pi_{t+1}$

Пусть $c_t(a)$ - количество раз, которое агент выбрал действие $a$ к моменту $t$
$$c_t(a) = \sum_i^{t}[a_i = a]$$
Тогда задачей агента ялвяется минимизация сожаления к моменту $T$, а именно
$$T\cdot\mu^* - \sum_a \mu_a \mathbb{E}[c_T(a)]$$
* $\mu_a$ - мат. ожидание награды за действие $a$
* $\mu^* = \max_a \mu_a$ - мат. ожидание оптимального действия

Так же можно встретить следующее определение сожаления:
$$T\cdot\mu^* - \sum_{t=1}^{T} r_t,$$
то есть в правой части стоит не ожидаемый выигрыш агента, а его конкретная реализация.

Впервые задача была предложена в этой [статье](http://projecteuclid.org/download/pdf_1/euclid.bams/1183517370).
### Модельная задача
1. $|A| = 100, T = 1000$
2. $\mu_a \sim \mathcal{N}(0, 1)$
3. $p(r|a) = \mathcal{N}(r; \mu_a, 1)$

Симуляция игры запускается $10^4$ раз и строится график в осях *номер шага в игре* $\times$ *усредненная сумарная премия к текущему шагу*.

### Известные стратегии
Введем следущие определения:
* $Q_t(a)$ - средняя премия действия $a$ к раунду $t$, при росте $c_t(a)$ стремиться к $\mu_a$.
* $Q^*(a) = \lim_{t \rightarrow \infty} Q_t(a)$ -- ценность действия $a$.
* $A_t = \arg\max_a Q_t(a)$ -- множество действий, которое имеет максимальную среднюю премию к рануду $t$.

**Задание**
1. Можно ли вычилсить $Q_{t+1}(a)$ инкерементально (известно лишь  $Q_t(a)$ и награда  $r_{t+1}$, назначенная за выбор действия $a$).
2. Используйте этот подход далее.

#### Жадная стратегия (greedy policy)
Стратегия $\pi_t$ заключается в том, что мы равновероятно выбираем $a$ из $A_t$
$$\pi_t(a)= \frac{1}{|A_t|}[a \in A_t]$$
**Задание**
1. Реализуйте данную стратегию
2. Опишите, в чем ее главный недостаток
3. Как вы инициализировали $\pi_1$?
4. Нужно ли несколько стартовых игр, чтобы инициализировать $\pi_1$?

#### $\varepsilon$-жадная стратегия ($\varepsilon$-greedy policy)
Проблема предыдущего подхода в том, что он лишь жадным образом пытается эксплуатировать среду. Однако какое-то время необходимо тратить также на ее изучение, чтобы максимизировать премию в долгосрочном периоде. Таким образом идет речь о балансе *exploration/exploitation*.

Возможная модификация предыдущего подхода:
$$\pi_t(a)=\frac{1-\varepsilon}{|A_t|}[a \in A_t] + \frac{\varepsilon}{|A|}, \varepsilon \in [0, 1]$$



**Задание**
1. Реализуйте данную стратегию
2. Что происходит с ростом $\varepsilon$?
3. Как бы изменяли $\varepsilon$ по мере обучения агента?
4. Опробуйте жадную и $\varepsilon$-жадную стратегии на модельной задаче. Попробуйте разные $\varepsilon$.

#### Softmax

Другая интерпретация $\varepsilon$-жадной стратегии
$$\pi_t(a)=\frac{\exp(\frac{1}{\varepsilon} \cdot Q_t(a))}{\sum_{b} \exp(\frac{1}{\varepsilon} \cdot Q_t(b))}, \varepsilon > 0$$

**Задание**
1. Реализуйте данную стратегию
2. Что происходит если $\varepsilon$ стремится к $0$? А бесконечности?
3. Сравните softmax и $\varepsilon$-жадную стратегии на модельной задаче.

#### Метод UCB (upper confidence bound)
Предложенный ниже алгоритм в некотором смысле является оптимальным, подробнее об этом [здесь]( http://homes.di.unimi.it/~cesabian/Pubblicazioni/ml-02.pdf). Для каждого момента $t$ он определяет множество наиболее потенциально выгодных действий. Выбор $a$ из $A_t$ происходит равновероятно.

$$A_t = \arg\max Q_t(a) + \varepsilon \sqrt{\frac{2 \ln t}{c_t(a)}}, \varepsilon \geq 0$$

Первая часть нам уже знакома, а вот вторую часть можно воспринимать, как величину, показывающую на сколько точна наша оценка $Q_t(a)$. Стратегия сама пытается найти баланс между *exploration/exploitation*.

**Задание**
1. Реализуйте данную стратегию
2. Что происходит если $\varepsilon$ увеличивается?
3. На модельной задаче сравнените лучшую версию $softmax$, $\varepsilon$-жадную стратегии и UCB метод.

#### Градиентный метод (gradient bandit policy)

Существует еще так называемые адаптивные стратегии. Они могут быть использованы, если среда не является стационарной (распредление премий могу меняться). В таком случае предлагается использовать уже знакомое нам экспоненциальное сглаживание.

Мы подсчитываем среднюю сглаженную премию к моменту $t$ по всем действиям ($\alpha$ регулируте глубину истории):
$$\bar{r}_{t+1} = (1-\alpha_t)\cdot\bar{r}_t(a)+\alpha_t r_{t+1} = \bar{r}_t +\alpha_t (r_{t+1}-\bar{r}_t(a)), \alpha \in [0, 1]$$

Для каждого действия у нас есть приоритет $p_t(a)$. После очередного шага идет корректировка с шагом $\lambda$. Если было выбрано действие $a$, то
$$p_{t+1}(a) = p_t(a)+\lambda(r_t-\bar{r}_t)(1-p_t(a))$$
в ином случае
$$p_{t+1}(a) = p_t(a)+\lambda(r_t-\bar{r}_t)p_t(a)$$

Тогда стратегия на момент $\pi_{t+1}$ будет выглядеть так
$$\pi_{t+1}(a) = \frac{\exp(p_{t+1}(a))}{\sum_{b} \exp(p_{t+1}(b))}$$
Подробно обоснование метода вы можете найти в книге.


**Задание**
1. Реализуйте данную стратегию.
2. Покажите какая из описанных моделей ведет себя лучше на модельной задаче.

### Дальнейшее чтение
Существуют также задача многорукого бандита с контекстом, формальная постановка выглядит следующим образом.
* $A$ - множество допустимых действий
* $X$ - пространство контекстов среды
* $p(r|a, x)$ - распределение премии для действия $a$ в условиях контекста $x$
* $\pi_t(a|x)$ - стратегия агента на момент $t$ в условиях контекста $x$.

Пример использования такой модели при показе новостей [здесь](http://www.research.rutgers.edu/~lihong/pub/Li10Contextual.pdf).